# Resting State - Classical Features - KMeans Clustering

## 0. Imports & Constants

### Libraries

In [ ]:
import os
import sys
import joblib
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.model_selection import LeaveOneOut, RandomizedSearchCV
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.base import clone

import umap.umap_ as umap

In [ ]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

from FeatureExtraction import FeatureExtractor, FEUtilz
from ClusteringUtilz import *

### Constans

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
FEATURES_DATA_PATH = rf"..\..\results\Resting State - best\resting_state_feature_table_22_05_2025_16_25.csv"
CLINICAL_DATA_PATH = rf'C:\Users\97254\Projects Data\ClusteringPD\cluestring_pd_clinical_data.xlsx'

CLINICAL_FEATURE_NAMES = ['Age', 'Disease duration', 
                        'LEDD', 'MoCA', 'UPDRS Total', 'UPDRS part III', 
                        'Gait speed usual', 'Gait speed DT', 
                        'CTT1', 'CTT2']

MODEL_FEATURE_NAMES = [
    'LZC_mean', 'PermEn_mean', 'fooof_exponent', 
    'beta_relative_bandpower', 'gamma_relative_bandpower', 'alpha_relative_bandpower', 'theta_relative_bandpower'
]

N_SEARCHES = 100
RANDOM_STATE = 42

### Import Data

In [ ]:
feature_table = pd.read_csv(FEATURES_DATA_PATH)
feature_table = feature_table.loc[feature_table['FeatureName'].isin(MODEL_FEATURE_NAMES)]
feature_table['Task'] = feature_table['Condition'].str.split('_').str[0]
feature_table['Condition'] = feature_table['Condition'].str.split('_').str[1]

In [ ]:
feature_table['FeatureName'].unique()

In [ ]:
data, subjects, raw_feature_names = FEUtilz.get_feature_matrix(feature_table, normalize=True, agg_regions=True)

In [ ]:
PD_mask = pd.Series(subjects).str.startswith('PD')

PD_data = data[PD_mask, :]
HC_data = data[~PD_mask, :]

PD_subjects = subjects[PD_mask]

In [ ]:
PD_data.shape

### A Vanilla Model

In [ ]:
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
vanilla_kmeans = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('KMeans', KMeans(n_clusters=3))
])

In [ ]:
evaluator = ClusterReport(models=[vanilla_kmeans], model_names=['Vanilla Model'], cl_data_path=CLINICAL_DATA_PATH, cl_fnames=CLINICAL_FEATURE_NAMES
                       , PD_data=PD_data, PD_subjects=PD_subjects, HC_data=HC_data)

In [ ]:
evaluator.report()

## 1. Hyperparameter Tuning

In [ ]:
def silhouette_scorer(estimator, X, y=None):
    Xp = estimator[:-1].transform(X)
    labels = estimator.predict(X)
    return silhouette_score(Xp, labels)

In [ ]:
cv = LeaveOneOutTrainEval()

### Vanilla-KMeans

#### K = 2

In [ ]:
vanilla_kmeans_2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('KMeans', KMeans(n_clusters=2))
])

vanilla_kmeans_2.fit(PD_data)

#### K = 3

In [ ]:
vanilla_kmeans_3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('KMeans', KMeans(n_clusters=3))
])

vanilla_kmeans_3.fit(PD_data)

### PCA-KMeans

#### K = 2

In [ ]:
PCA_kmeans_2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('PCA', PCA()),
    ('KMeans', KMeans(n_clusters=2))
])

PCA_params = {
    'PCA__n_components': [0.7, 0.8, 0.9, 0.95]
}

PCA_kmeans_grid_2 = RandomizedSearchCV(
    PCA_kmeans_2,
    PCA_params,
    scoring=silhouette_scorer,
    cv=cv,
    n_iter=N_SEARCHES,
    verbose=1
)

In [ ]:
PCA_kmeans_grid_2.fit(PD_data)

In [ ]:
PCA_kmeans_grid_2.best_estimator_.named_steps['PCA'].n_components_

In [ ]:
evaluator.set_models(models=[PCA_kmeans_grid_2.best_estimator_, vanilla_kmeans_2], model_names=['PCA', 'Vanilla'])
evaluator.report(include_clinical=False)

#### K = 3

In [ ]:
PCA_kmeans_3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('PCA', PCA()),
    ('KMeans', KMeans(n_clusters=3))
])

PCA_kmeans_grid_3 = RandomizedSearchCV(
    PCA_kmeans_3,
    PCA_params,
    scoring=silhouette_scorer,
    cv=cv,
    n_iter=N_SEARCHES,
    verbose=1
)

In [ ]:
PCA_kmeans_grid_3.fit(PD_data)

In [ ]:
PCA_kmeans_grid_3.best_estimator_.named_steps['PCA'].n_components_

In [ ]:
evaluator.set_models(models=[PCA_kmeans_grid_3.best_estimator_, vanilla_kmeans_3], model_names=['PCA', 'Vanilla'])
evaluator.report(include_clinical=False)

### UMAP-KMeans

#### K = 2

In [ ]:
UMAP_kmeans_2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('UMAP', umap.UMAP()),
    ('KMeans', KMeans(n_clusters=2))
])

UMAP_params = {
    'UMAP__n_components': np.arange(3, 22, 3),
    'UMAP__n_neighbors': np.arange(5, 21, 5)
}

UMAP_kmeans_grid_2 = RandomizedSearchCV(
    UMAP_kmeans_2,
    UMAP_params,
    scoring=silhouette_scorer,
    cv=cv,
    n_iter=N_SEARCHES,
    verbose=1
)

In [ ]:
UMAP_kmeans_grid_2.fit(PD_data)

In [ ]:
evaluator.set_models(models=[UMAP_kmeans_grid_2.best_estimator_, vanilla_kmeans_2], model_names=['UMAP', 'Vanilla'])
evaluator.report(include_clinical=False)

#### K = 3

In [ ]:
UMAP_kmeans_3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('UMAP', umap.UMAP()),
    ('KMeans', KMeans(n_clusters=3))
])

UMAP_kmeans_grid_3 = RandomizedSearchCV(
    UMAP_kmeans_3,
    UMAP_params,
    scoring=silhouette_scorer,
    cv=cv,
    n_iter=N_SEARCHES,
    verbose=1
)

In [ ]:
UMAP_kmeans_grid_3.fit(PD_data)

In [ ]:
evaluator.set_models(models=[UMAP_kmeans_grid_3.best_estimator_, vanilla_kmeans_3], model_names=['UMAP', 'Vanilla'])
evaluator.report(include_clinical=False)

## 2. Best Model Training & Evaluation

### General Evaluation

In [ ]:
best_model = UMAP_kmeans_grid_3.best_estimator_
best_model.named_steps['UMAP'].random_state = RANDOM_STATE
best_model.named_steps['KMeans'].random_state = RANDOM_STATE

joblib.dump(best_model, rf'Models\UMAP_KMeans.pkl')

evaluator.set_models(models=[best_model], names=['UMAP-KMeans [K=3]'])
evaluator.report()